<a href="https://colab.research.google.com/github/JorgeAlfaroAguirre/JorgeAlfaroAguirre/blob/main/MineriaJorgeAlfaro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install gradio pandas numpy matplotlib seaborn


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import seaborn as sns

# Variable global para almacenar el DataFrame cargado
dataframe = None

# --- Cargar archivo ---
def process_file(file):
    global dataframe
    if file.name.endswith('.csv'):
        dataframe = pd.read_csv(file.name)
    elif file.name.endswith(('.xls', '.xlsx')):
        dataframe = pd.read_excel(file.name)
    else:
        return "Error: Solo se aceptan archivos CSV o Excel.", None

    headers = dataframe.columns.tolist()

    numeric_headers = [col for col in headers if pd.api.types.is_numeric_dtype(dataframe[col])]
    categorical_headers = [col for col in headers if not pd.api.types.is_numeric_dtype(dataframe[col])]

    total_rows = len(dataframe)
    summary = (
        f"Archivo cargado correctamente.\n\n"
        f"Headers numéricos:\n" + "\n".join(numeric_headers) + "\n\n"
        f"Headers categóricos:\n" + "\n".join(categorical_headers) + "\n\n"
        f"Número total de filas: {total_rows}"
    )
    return summary, dataframe.head(100)

# --- Limpieza de valores mixtos ---
def remove_mixed_data():
    global dataframe
    if dataframe is None:
        return "No hay un archivo cargado para limpiar.", None, None

    original_rows = len(dataframe)
    rows_to_drop = set()

    for column in dataframe.columns:
        valid_values = dataframe[column].dropna().apply(lambda x: x if len(str(x).strip()) > 0 else None).dropna()
        unique_types = valid_values.apply(type).unique()

        if len(unique_types) > 1:
            numeric_count = valid_values.apply(lambda x: isinstance(x, (int, float))).sum()
            categorical_count = valid_values.apply(lambda x: isinstance(x, str)).sum()

            if numeric_count >= categorical_count:
                dataframe[column] = dataframe[column].apply(
                    lambda x: pd.to_numeric(x, errors='coerce') if isinstance(x, (int, float, str)) else x
                )
            else:
                dataframe[column] = dataframe[column].apply(
                    lambda x: x if isinstance(x, str) else None
                )

            rows_to_drop.update(dataframe[dataframe[column].notnull() & dataframe[column].isnull()].index)

    dataframe.drop(index=rows_to_drop, inplace=True)
    affected_rows = len(rows_to_drop)

    numeric_headers = [col for col in dataframe.columns if pd.api.types.is_numeric_dtype(dataframe[col])]
    categorical_headers = [col for col in dataframe.columns if not pd.api.types.is_numeric_dtype(dataframe[col])]
    summary = (
        f"Headers actualizados después de limpiar valores mixtos:\n\n"
        f"Headers numéricos:\n" + "\n".join(numeric_headers) + "\n\n"
        f"Headers categóricos:\n" + "\n".join(categorical_headers) + "\n\n"
        f"Número total de filas: {len(dataframe)}"
    )

    return f"Se eliminaron {affected_rows} filas con valores mixtos.", summary, dataframe.head(100)

# --- Limpieza de valores nulos ---
def handle_nulls(option):
    global dataframe
    if dataframe is None:
        return "No hay un archivo cargado para limpiar.", None

    affected_rows = dataframe.isnull().any(axis=1).sum()
    if option == "Dejar los nulos o vacíos con valor 0":
        dataframe.fillna(0, inplace=True)
        modification = "Valores nulos reemplazados por 0."
    elif option == "Borrar la fila completa":
        dataframe.dropna(inplace=True)
        modification = "Filas con valores nulos eliminadas."
    elif option == "Colocar el promedio":
        for column in dataframe.columns:
            if dataframe[column].isnull().any() and pd.api.types.is_numeric_dtype(dataframe[column]):
                mean_value = dataframe[column].mean()
                dataframe[column].fillna(mean_value, inplace=True)
        modification = "Valores nulos reemplazados por el promedio."
    elif option == "Colocar la mediana":
        for column in dataframe.columns:
            if dataframe[column].isnull().any() and pd.api.types.is_numeric_dtype(dataframe[column]):
                median_value = dataframe[column].median()
                dataframe[column].fillna(median_value, inplace=True)
        modification = "Valores nulos reemplazados por la mediana."
    else:
        return "Opción no válida.", None

    return f"{modification}\nRegistros afectados: {affected_rows}", dataframe.head(100)

# --- Normalización y estandarización ---
def normalize_or_standardize(column, method):
    global dataframe
    if dataframe is None:
        return "No hay un archivo cargado para analizar.", None

    if column not in dataframe.columns or not pd.api.types.is_numeric_dtype(dataframe[column]):
        return "Alerta: Debes seleccionar una columna numérica.", None

    if method == "Min-Max":
        min_val = dataframe[column].min()
        max_val = dataframe[column].max()
        dataframe[column] = (dataframe[column] - min_val) / (max_val - min_val)
        result = f"Normalización Min-Max realizada en la columna {column}."
    elif method == "Z-Score":
        mean = dataframe[column].mean()
        std_dev = dataframe[column].std()
        dataframe[column] = (dataframe[column] - mean) / std_dev
        result = f"Estandarización Z-Score realizada en la columna {column}."
    else:
        return "Método no válido.", None

    return result, dataframe.head(100)

# --- Análisis estadístico ---
def analyze_statistics(columns):
    """Realiza un análisis estadístico básico en las columnas seleccionadas."""
    global dataframe
    if dataframe is None:
        return "Error: No hay un archivo cargado para analizar."

    # Dividir las columnas por comas y quitar espacios en blanco
    selected_columns = [col.strip() for col in columns.split(",")]

    # Verificar si las columnas existen y son numéricas
    invalid_columns = [col for col in selected_columns if col not in dataframe.columns or not pd.api.types.is_numeric_dtype(dataframe[col])]
    if invalid_columns:
        return f"Error: Las siguientes columnas no son válidas o no son numéricas: {', '.join(invalid_columns)}"

    try:
        # Correlaciones entre las columnas seleccionadas
        correlation_matrix = dataframe[selected_columns].corr().to_string()

        # Curtosis para cada columna seleccionada
        kurtosis_values = dataframe[selected_columns].kurt().to_dict()

        # Simetría (Skewness) para cada columna seleccionada
        skewness_values = dataframe[selected_columns].skew().to_dict()

        # Outliers (IQR) para cada columna seleccionada
        outlier_info = {}
        for col in selected_columns:
            Q1 = dataframe[col].quantile(0.25)
            Q3 = dataframe[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = dataframe[(dataframe[col] < lower_bound) | (dataframe[col] > upper_bound)][col]
            outlier_info[col] = {
                "Número de outliers": len(outliers),
                "Valores": outliers.tolist()
            }

        # Generar resultados
        result = "--- Análisis Estadístico ---\n"
        result += "--- Correlaciones ---\n" + correlation_matrix + "\n\n"
        result += "--- Curtosis ---\n" + "\n".join([f"{col}: {round(val, 2)}" for col, val in kurtosis_values.items()]) + "\n\n"
        result += "--- Simetría ---\n" + "\n".join([f"{col}: {round(val, 2)}" for col, val in skewness_values.items()]) + "\n\n"
        result += "--- Outliers (IQR) ---\n"
        for col, info in outlier_info.items():
            result += f"Columna: {col}\nNúmero de outliers: {info['Número de outliers']}\nValores: {info['Valores']}\n\n"

        return result
    except Exception as e:
        return f"Error durante el análisis estadístico: {str(e)}"


In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import seaborn as sns

# Variable global para almacenar el DataFrame cargado
dataframe = None

# --- Graficos y Conclusiones ---
def generate_graphs_and_conclusions(column_inputs):
    """Genera gráficos y un informe basado en columnas seleccionadas."""
    global dataframe
    if dataframe is None:
        return "Error: No hay un archivo cargado.", None, None, None, None, None

    selected_columns = {key: [col.strip() for col in cols.split(",") if col.strip()] for key, cols in column_inputs.items()}
    outputs = {
        "boxplot": None,
        "heatmap": None,
        "histograms": [],
        "density": [],
        "conclusions": []
    }

    try:
        conclusions = []

        # --- Boxplot ---
        if "boxplot" in selected_columns and selected_columns["boxplot"]:
            plt.figure(figsize=(10, 6))
            dataframe[selected_columns["boxplot"]].boxplot()
            plt.title("Boxplot de Outliers")
            boxplot_buffer = BytesIO()
            plt.savefig(boxplot_buffer, format="png")
            plt.close()
            boxplot_buffer.seek(0)
            outputs["boxplot"] = boxplot_buffer
            conclusions.append("Boxplot generado correctamente.")

        # --- Mapa de Correlaciones ---
        if "heatmap" in selected_columns and len(selected_columns["heatmap"]) >= 2:
            corr_matrix = dataframe[selected_columns["heatmap"]].corr()
            plt.figure(figsize=(10, 8))
            sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
            plt.title("Mapa de Correlaciones")
            heatmap_buffer = BytesIO()
            plt.savefig(heatmap_buffer, format="png")
            plt.close()
            heatmap_buffer.seek(0)
            outputs["heatmap"] = heatmap_buffer
            conclusions.append("Mapa de correlaciones generado correctamente.")
        else:
            conclusions.append(
                "Nota: No se pudo generar el mapa de correlaciones porque se necesitan al menos 2 columnas."
            )

        # --- Histograma ---
        if "histogram" in selected_columns and selected_columns["histogram"]:
            for col in selected_columns["histogram"]:
                plt.figure(figsize=(10, 6))
                sns.histplot(dataframe[col].dropna(), bins=20, kde=False, color="skyblue")
                plt.title(f"Histograma de {col}")
                plt.xlabel("Valores")
                plt.ylabel("Frecuencia")
                hist_buffer = BytesIO()
                plt.savefig(hist_buffer, format="png")
                plt.close()
                hist_buffer.seek(0)
                outputs["histograms"].append(hist_buffer)
                conclusions.append(f"Histograma generado para la columna '{col}'.")

        # --- Gráfico de Densidad (KDE) ---
        if "density" in selected_columns and selected_columns["density"]:
            for col in selected_columns["density"]:
                plt.figure(figsize=(10, 6))
                sns.kdeplot(data=dataframe[col].dropna(), fill=True, color="skyblue")
                plt.title(f"Densidad de {col}")
                plt.xlabel("Valores")
                plt.ylabel("Densidad")
                density_buffer = BytesIO()
                plt.savefig(density_buffer, format="png")
                plt.close()
                density_buffer.seek(0)
                outputs["density"].append(density_buffer)
                conclusions.append(f"Gráfico de densidad generado para la columna '{col}'.")

        # --- Conclusiones sobre Curtosis ---
        if "boxplot" in selected_columns and selected_columns["boxplot"]:
            for col in selected_columns["boxplot"]:
                curtosis = dataframe[col].kurt()
                if curtosis > 3:
                    conclusions.append(
                        f"La columna '{col}' tiene una curtosis alta (leptocúrtica), lo que indica colas más pesadas."
                    )
                elif curtosis < 3:
                    conclusions.append(
                        f"La columna '{col}' tiene una curtosis baja (platicúrtica), lo que indica colas más ligeras."
                    )
                else:
                    conclusions.append(
                        f"La columna '{col}' tiene una curtosis cercana a 3 (mesocúrtica), lo que indica una distribución normal."
                    )

        outputs["conclusions"] = conclusions
        return "Gráficos generados correctamente.", outputs["boxplot"], outputs["heatmap"], outputs["histograms"], outputs["density"], "\n".join(conclusions)

    except Exception as e:
        return f"Error al generar gráficos: {str(e)}", None, None, None, None, None

# --- Exportar a Excel ---
def export_to_excel(file_name="analisis.xlsx"):
    """Exporta los resultados del análisis a un archivo Excel con múltiples hojas."""
    global dataframe
    if dataframe is None:
        return "Error: No hay datos cargados para exportar."

    try:
        with pd.ExcelWriter(file_name, engine="openpyxl") as writer:
            # Datos originales
            dataframe.to_excel(writer, sheet_name="Datos Originales", index=False)

            # Resumen estadístico
            stats = dataframe.describe()
            stats.to_excel(writer, sheet_name="Resumen Estadístico")

        return f"Archivo Excel exportado con éxito: {file_name}"
    except Exception as e:
        return f"Error al exportar a Excel: {str(e)}"


In [40]:
import gradio as gr

# --- Interfaz de usuario ---
with gr.Blocks() as interface:
    # Encabezado de la aplicación
    gr.Markdown("## Aplicación Interactiva para Procesamiento y Análisis de Datos")
    gr.Markdown(
        "### Desarrollado por Jorge Alfaro\n"
        "Esta herramienta fue creada para facilitar el procesamiento y análisis de datos de forma interactiva, "
        "integrando técnicas de limpieza, transformación y análisis estadístico."
    )

    # Carga de archivo
    gr.Markdown(
        "### Carga de archivo\n"
        "Permite cargar un archivo en formato CSV o Excel para su análisis. "
        "El sistema genera un resumen indicando las columnas numéricas, categóricas y el número de filas totales."
    )
    file_input = gr.File(label="Cargar archivo")
    summary_output = gr.Textbox(label="Resumen de headers (Numéricos y Categóricos)")
    table_output = gr.DataFrame(label="Vista previa (hasta 100 filas)")
    file_input.change(process_file, inputs=[file_input], outputs=[summary_output, table_output])

    # Limpieza de valores mixtos
    gr.Markdown(
        "### Limpieza de Valores Mixtos\n"
        "Los valores mixtos se producen cuando una columna numérica contiene datos no numéricos, dificultando el análisis. "
        "Esta función elimina esos valores no numéricos para normalizar los datos."
    )
    remove_mixed_button = gr.Button("Eliminar valores mixtos")
    remove_mixed_result = gr.Textbox(label="Resultado de limpieza de valores mixtos")
    updated_summary_output = gr.Textbox(label="Resumen actualizado de headers")
    updated_table_output = gr.DataFrame(label="Datos después de limpiar valores mixtos")
    remove_mixed_button.click(
        remove_mixed_data,
        outputs=[remove_mixed_result, updated_summary_output, updated_table_output]
    )

    # Limpieza de datos nulos
    gr.Markdown(
        "### Limpieza de Valores Nulos\n"
        "Los valores nulos son datos faltantes en las columnas. "
        "Esta función permite elegir entre reemplazarlos con 0, la media, la mediana, o eliminar las filas completas."
    )
    null_handling_options = gr.Dropdown(
        choices=["Dejar los nulos o vacíos con valor 0", "Borrar la fila completa", "Colocar el promedio", "Colocar la mediana"],
        label="Selecciona una acción para los valores nulos"
    )
    clean_button = gr.Button("Aplicar limpieza")
    clean_result = gr.Textbox(label="Resultado de limpieza")
    clean_table = gr.DataFrame(label="Datos después de la limpieza")
    clean_button.click(handle_nulls, inputs=[null_handling_options], outputs=[clean_result, clean_table])

    # Selección de headers y transformación
    gr.Markdown(
        "### Selección de Headers y Transformaciones\n"
        "Permite seleccionar una columna numérica para realizar transformaciones como normalización (Min-Max) o estandarización (Z-Score)."
    )
    header_selector = gr.Textbox(label="Escribe el nombre del header (columna numérica)", interactive=True)

    # Normalización y estandarización
    gr.Markdown(
        "### Normalización y Estandarización\n"
        "#### Normalización (Min-Max):\n"
        "- Escala los valores entre 0 y 1.\n"
        "- Fórmula: `(valor - mínimo) / (máximo - mínimo)`.\n\n"
        "#### Estandarización (Z-Score):\n"
        "- Ajusta los valores para que tengan una media de 0 y una desviación estándar de 1.\n"
        "- Fórmula: `(valor - media) / desviación estándar`."
    )
    method_selector = gr.Dropdown(choices=["Min-Max", "Z-Score"], label="Selecciona el método de transformación")
    normalize_button = gr.Button("Aplicar transformación")
    normalize_result = gr.Textbox(label="Resultado de la transformación")
    normalize_table = gr.DataFrame(label="Datos después de transformación")
    normalize_button.click(
        normalize_or_standardize,
        inputs=[header_selector, method_selector],
        outputs=[normalize_result, normalize_table]
    )

    # Campo para seleccionar columnas para gráficos y análisis estadístico
    gr.Markdown(
        "### Generación de Gráficos y Conclusiones\n"
        "Selecciona las columnas específicas para generar los diferentes gráficos. "
        "El sistema generará las conclusiones con base en la curtosis y otros análisis estadísticos.\n"
        "- **Boxplot**: Admite 1 o más columnas.\n"
        "- **Mapa de Correlaciones**: Requiere 2 o más columnas.\n"
        "- **Histograma y Densidad**: Requieren 1 columna."
    )
    boxplot_input = gr.Textbox(label="Columnas para Boxplot (separadas por comas)")
    heatmap_input = gr.Textbox(label="Columnas para Mapa de Correlaciones (2 o más columnas)")
    histogram_input = gr.Textbox(label="Columnas para Histograma (1 columna)")
    density_input = gr.Textbox(label="Columnas para Densidad (1 columna)")
    generate_graphs_button = gr.Button("Generar Gráficos")
    graphs_boxplot = gr.Image(label="Boxplot de Outliers")
    graphs_correlation = gr.Image(label="Mapa de Correlaciones")
    graphs_histograms = gr.Gallery(label="Histogramas")
    graphs_density = gr.Gallery(label="Gráficos de Densidad")
    conclusions_output = gr.Textbox(label="Conclusiones Generales")
    generate_graphs_button.click(
        generate_graphs_and_conclusions,
        inputs=[boxplot_input, heatmap_input, histogram_input, density_input],
        outputs=[graphs_boxplot, graphs_correlation, graphs_histograms, graphs_density, conclusions_output]
    )

    # Generar Informe
    gr.Markdown(
        "### Generación de Informes\n"
        "Genera un archivo de texto con un resumen de las decisiones tomadas y un análisis estadístico de los datos cargados."
    )
    generate_log_button = gr.Button("Generar Informe")
    log_file = gr.File(label="Descargar Informe")
    generate_log_button.click(
        lambda: "informe_log.txt", outputs=log_file
    )

    # Exportar a Excel
    gr.Markdown(
        "### Exportar a Excel\n"
        "Exporta los datos originales, análisis estadístico, y otros resultados en un archivo Excel con múltiples hojas."
    )
    export_button = gr.Button("Exportar a Excel")
    excel_file = gr.File(label="Descargar Excel")
    export_button.click(
        lambda: "analisis.xlsx", outputs=excel_file
    )

interface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:999: UserWarning: Expected 1 arguments for function <function generate_graphs_and_conclusions at 0x7ac2098cad40>, received 4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1007: UserWarning: Expected maximum 1 arguments for function <function generate_graphs_and_conclusions at 0x7ac2098cad40>, received 4.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2df0ae05a392da1cfb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
